Import the necessary dependent libraries

In [1]:
import os
from module import MLPModule
from utils import aggregate_weights, save_global_measure, load_client_data, calculate_global_measures
from tqdm import tqdm
import torch
from client import Client

Set the necessary parameters and hyperparameters for the experiment and model

In [2]:
seed = 54
dataset_name = "n-baiot"
num_rounds = 25
input_dim = 0

if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps:0")
else:
    device = torch.device("cpu")

hyperparameters = {
    'input_dim': input_dim,
    'lr': 0.001,
    'hidden_neurons_num': 512,
    'batch_size': 128,
    'seed': seed,
    'device': device
}

In [3]:
client_data_list = load_client_data(os.path.join(dataset_name, "split"), device)

In [4]:
def process_client_data(data_list, keep_columns):
    processed_data_list = []

    for client_data in data_list:
        X_train, Y_train = client_data[0]
        processed_X_train = X_train[:, keep_columns]
        processed_train = (processed_X_train, Y_train)

        X_val, Y_val = client_data[1]
        processed_X_val = X_val[:, keep_columns]
        processed_val = (processed_X_val, Y_val)

        X_test, Y_test = client_data[2]
        processed_X_test = X_test[:, keep_columns]
        processed_test = (processed_X_test, Y_test)

        processed_client_data = (processed_train, processed_val, processed_test)
        processed_data_list.append(processed_client_data)

    return processed_data_list

In [5]:
columns_to_keep = [0, 2, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 28, 29, 38, 39]
if dataset_name == "n-baiot":
    columns_to_keep = [0, 3, 6, 30, 37, 44, 31, 38, 45, 32, 39, 46, 60, 63, 66, 75, 82, 89, 76, 83, 90]
client_data_list = process_client_data(client_data_list, columns_to_keep)
hyperparameters['input_dim'] = len(columns_to_keep)

Generate clients

In [6]:
def create_clients(client_data, hyperparameter):
    client_list = []
    for i, (train_set, test_set, val_set) in enumerate(client_data):
        c = Client(
            id=i,
            hyperparameters=hyperparameter,
            train_set=train_set,
            test_set=test_set,
            val_set=val_set
        )
        client_list.append(c)
    return client_list
clients = create_clients(client_data_list, hyperparameters)

FedAvg training process

In [7]:
# Initialize model
global_model = MLPModule(hyperparameters)

# Pre-trained
model_weights_list = []
client_n_samples_list = {}
for client in clients:
    client.receive_global_model(global_model)
    client_model_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
    client_n_samples_list[client.id] = client_n_samples
    model_weights_list.append(client_model_parameters)
    
global_model.set_parameters(aggregate_weights(model_weights_list, client_n_samples_list))

In [8]:
# Record training results
record_global_train_measures = [None] * num_rounds
record_global_val_measures = [None] * num_rounds

pbar = tqdm(total=num_rounds)
# Perform num_rounds rounds of training
for idx in range(num_rounds):
    model_weights_list = []
    client_n_samples_list = {}
    all_client_train_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }
    all_client_val_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }
    
    for client in clients:
        # Client receives new global model
        client.receive_global_model(global_model)

        # The client trains the model locally
        client_model_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
        client_n_samples_list[client.id] = client_n_samples
        model_weights_list.append(client_model_parameters)
        for key in all_client_train_measures:
            all_client_train_measures[key][client.id] = (client_measures['train'][key])
            all_client_val_measures[key][client.id] = (client_measures['val'][key])

    global_train_measures, global_val_measures = calculate_global_measures(clients,
                                                                           all_client_train_measures,
                                                                           all_client_val_measures,
                                                                           display_result=True)
    record_global_train_measures[idx] = global_train_measures
    record_global_val_measures[idx] = global_val_measures

    pbar.update(1)

    # Aggregate new global model
    global_model.set_parameters(aggregate_weights(model_weights_list, client_n_samples_list))
    
pbar.reset()

  4%|▍         | 1/25 [00:40<16:07, 40.32s/it]

Global Train Set Loss: 0.3245374543008779
Global Train Set Accuracy: 0.8512858724198814
Global Train Set FPR: 0.580530196545936
Global Train Set TPR: 0.8561349479387258
Global Train Set BER: 0.3571976243036049
Global Validation Set Loss: 0.20218206264819666
Global Validation Set Accuracy: 0.8896984106989878
Global Validation Set FPR: 0.34437682093677296
Global Validation Set TPR: 0.8743494384217513
Global Validation Set BER: 0.1900136912575108


  8%|▊         | 2/25 [01:20<15:24, 40.21s/it]

Global Train Set Loss: 0.2847062147513999
Global Train Set Accuracy: 0.9021369464907845
Global Train Set FPR: 0.48741504999435165
Global Train Set TPR: 0.8372423002624305
Global Train Set BER: 0.3200863748659607
Global Validation Set Loss: 0.1515230894777387
Global Validation Set Accuracy: 0.9432002210104682
Global Validation Set FPR: 0.20128610347914144
Global Validation Set TPR: 0.8955144610440631
Global Validation Set BER: 0.10788582121753913


 12%|█▏        | 3/25 [02:01<14:53, 40.63s/it]

...

Global Train Set Loss: 0.2698731307750343
Global Train Set Accuracy: 0.9174765802523787
Global Train Set FPR: 0.46057171784117
Global Train Set TPR: 0.8313701948377787
Global Train Set BER: 0.30960076150169547
Global Validation Set Loss: 0.1360894244675942
Global Validation Set Accuracy: 0.9655199079116893
Global Validation Set FPR: 0.15351055997263147
Global Validation Set TPR: 0.9072288487420355
Global Validation Set BER: 0.07814085561529821


 96%|█████████▌| 24/25 [16:23<00:40, 40.37s/it]

Global Train Set Loss: 0.2695671130087738
Global Train Set Accuracy: 0.9171365041632137
Global Train Set FPR: 0.4605950282537839
Global Train Set TPR: 0.8279835615213393
Global Train Set BER: 0.31130573336622236
Global Validation Set Loss: 0.13573431949817677
Global Validation Set Accuracy: 0.9656930586367203
Global Validation Set FPR: 0.15246560498089287
Global Validation Set TPR: 0.9072335421656093
Global Validation Set BER: 0.07761603140764191


  0%|          | 0/25 [00:00<?, ?it/s]         

Global Train Set Loss: 0.2691223174125145
Global Train Set Accuracy: 0.9171483345249776
Global Train Set FPR: 0.460502339819925
Global Train Set TPR: 0.8281471421072323
Global Train Set BER: 0.31117759885634655
Global Validation Set Loss: 0.1352994129072618
Global Validation Set Accuracy: 0.9662335754345753
Global Validation Set FPR: 0.1515168935718104
Global Validation Set TPR: 0.9072362405745185
Global Validation Set BER: 0.07714032649864602


Save experiment results

In [9]:
save_global_measure(record_global_train_measures, "train_measures_"+dataset_name+".csv", "FedTrust")
save_global_measure(record_global_val_measures, "val_measures_"+dataset_name+".csv", "FedTrust")

Saved measures to Experimental_results/FedTrust/train_measures_n-baiot.csv
Saved measures to Experimental_results/FedTrust/val_measures_n-baiot.csv


Test the performance of the model on the unseen test set

In [10]:
all_client_measures = {
    'accuracy': [],
    'fpr': [],
    'tpr': [],
    'ber': [],
    'loss': []
}
client_n_samples_list = {}
for client in clients:
    client.receive_global_model(global_model)
    client_measures, client_n_samples = client.evaluate_test_set()
    client_n_samples_list[client.id] = client_n_samples

    for key in all_client_measures:
        all_client_measures[key].append(client_measures[key])

global_measures = {}
for key in all_client_measures:
    global_measures[key] = sum([all_client_measures[key][c.id] * client_n_samples_list[c.id] for c in clients]) / sum(client_n_samples_list.values())

print("Global Test Set Loss:", global_measures['loss'])
print("Global Test Set Accuracy:", global_measures['accuracy'])
print("Global Test Set FPR:", global_measures['fpr'])
print("Global Test Set TPR:", global_measures['tpr'])
print("Global Test Set BER:", global_measures['ber'])

Global Test Set Loss: 0.0992762065479118
Global Test Set Accuracy: 0.9822230937664373
Global Test Set FPR: 0.18820480251049926
Global Test Set TPR: 0.9887116955918488
Global Test Set BER: 0.09441157017008425
